## Run SUMMA model ##

Run the SUMMA base model, with the model runs managed by Snakemake

Note that this may require updated


### Write snakemake file ###

This block of code writes the snakemake file

In [9]:
%%writefile ../rules/run_pysumma_snakemake.smk
"""

Snakemake file to run the base SUMMA simulations.

The model simulation is chunked by GRU to allow for parallelization on a cluster.

The chunks of GRUs are defined by the user

"""

from pathlib import Path
import sys
import pysumma as ps
# Import local packages
sys.path.append(str(Path('../').resolve()))

sys.path.append('../')
from scripts import gmet_to_summa_utils as gts_utils
config = gts_utils.resolve_paths(config,log_config = False)

# Resolve all file paths and directories in the config file
config['file_manager'] = '../../test_data/summa/settings/fileManager.txt'
config['summa_output_dir'] = '../../test_data/summa/output/'
config['attributes_nc'] = '../../test_data/summa/settings/attributes.nc'
config['gru_chunk_size'] = 3
config['case_name'] = 'tuolumne'
config['run_suffix'] = 'base'

# UPDATE LOCAL SUMMA PATH
config['summa_exe'] = '/Users/drc858/GitHub/summa/bin/summa.exe'


# Generate GRU start and count
num_grus = gts_utils.calc_num_grus(config['attributes_nc'])
gru_chunk_strings = gts_utils.generate_gru_start_and_count(num_grus, chunk_size=config['gru_chunk_size'])

rule run_summa_base_simulations:
    input:
        expand(Path(config['summa_output_dir'],f"{config['case_name']}_{config['run_suffix']}_{{gru_chunk}}_timestep.nc"),gru_chunk=gru_chunk_strings)

rule run_summa_in_gru_chunks:
    input:
        file_manager = Path(config['file_manager'])
    output:
        summa_chunked_output = Path(config['summa_output_dir'],f"{config['case_name']}_{config['run_suffix']}_{{gru_chunk}}_timestep.nc")
    params:
        summa_exe = config['summa_exe'],
        gru_start = lambda wildcards: gts_utils.extract_gru_int({wildcards.gru_chunk}),
        gru_count = config['gru_chunk_size']
    run:
        sim = ps.Simulation(params.summa_exe,input.file_manager)
        sim.run(run_suffix=config['run_suffix'], startGRU=params.gru_start, countGRU=params.gru_count, write_config=False)


        

Overwriting ../rules/run_pysumma_snakemake.smk


In [10]:
! snakemake --unlock -s ../rules/run_pysumma_snakemake.smk --configfile ../config/gmet_to_summa_test_case_config.yaml 
! snakemake -s ../rules/run_pysumma_snakemake.smk --cores 8 --configfile ../config/gmet_to_summa_test_case_config.yaml


Unlocking working directory.
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 8
Rules claiming more threads will be scaled down.
Job stats:
job                           count    min threads    max threads
--------------------------  -------  -------------  -------------
run_summa_base_simulations        1              1              1
run_summa_in_gru_chunks           9              1              1
total                            10              1              1

Select jobs to execute...

[Mon Aug 21 14:28:17 2023]
rule run_summa_in_gru_chunks:
    input: ../../test_data/summa/settings/fileManager.txt
    output: ../../test_data/summa/output/tuolumne_base_G19-21_timestep.nc
    jobid: 7
    reason: Missing output files: ../../test_data/summa/output/tuolumne_base_G19-21_timestep.nc
    wildcards: gru_chunk=G19-21
    resources: tmpdir=/var/folders/cf/h3xm_pj57nxdnm6lk83rzxt9jt0gpj/T

[Mon Aug 21 14:28:17 2023]
rule run_summa_in_gru_chunks:
    input: ../../test_data/su

In [30]:
%%writefile ../rules/run_fsca_simulations.smk
"""

Snakemake file to calculate fSCA based on SUMMA model simulations.

"""

from pathlib import Path
import sys
# Import local packages
sys.path.append(str(Path('../').resolve()))
sys.path.append(str(Path('../../').resolve()))
sys.path.append(str(Path('../../snow_dist').resolve()))
from scripts import ss_utils
from snow_dist import summa_simulation

# Resolve all file paths and directories in the config file
config['summa_output_dir'] = '/Users/drc858/Data/test/output'
config['summa_interim_dir'] = '/Users/drc858/Data/test/interim'
config['fsca_output_dir'] =  '/Users/drc858/Data/test/output/fsca'
config_file_path = '/Users/drc858/GitHub/snow_dist/settings/config_summa_model_tuolumne.yaml'

#Read first raw forcing file for easymore remapping
summa_result_filepaths = list(Path(config['summa_output_dir']).glob('*.nc'))
summa_filenames = [Path(filepath).name for filepath in summa_result_filepaths]
print(summa_filenames)

fsca_simulation = summa_simulation.SummaSimulation(config_file_path)

# Prepare summa model results for fsca
# This includes summing the input and output snowpack variables
rule run_fsca_model:
    input:
        expane(Path(config['summa_interim_dir'], "{filenames}")
rule prepare_fsca_model_input:
    input:
        summa_result = Path(config['summa_output_dir'],"{filenames}")
    output
        fsca_input = Path(config['summa_interim_dir'],"{filenames}")
    run:
        summa_ds = xr.open_dataset(input.input_summa_result)
        fsca_ds = summa_fsca_model.prepare_summa_fsca_input(summa_ds)
        fsca_ds.to_netcdf(output.fsca_input)

# Run fsca settings
rule run_fsca_simulations:
    input:
        prepped_files = expand(Path(config['summa_interim_dir'], "{filenames}"), filenames=summa_filenames)
    output:
        fsca_result = Path(config['fsca_output_dir'], "{filenames}")
    run:
        sim = summa_simulation.SummaSimulation(config)
        summa_fsca_model.run_summa_fsca_model(input.prepped_files,output.fsca_result)



Overwriting ../rules/run_fsca_simulations.smk


In [ ]:
-s ../rules/run_pysumma_snakemake.smk --configfile /Users/drc858/GitHub/summa_snakemake/snakemake/config/summa_snakemake_config_tuolumne.yaml 
! snakemake -s ../rules/run_pysumma_snakemake.smk --cores 8 --configfile /Users/drc858/GitHub/summa_snakemake/snakemake/config/summa_snakemake_config_tuolumne.yaml